In [1]:
import os
import h5py
import numpy as np
from IPython.lib.deepreload import reload
import matplotlib.pyplot as plt
from collections import OrderedDict
import skimage.io as io
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances
from matplotlib import cm
%matplotlib qt

In [2]:
import xrdmaptools
from xrdmaptools.XRDMap import XRDMap
from xrdmaptools.reflections.SpotModels import GaussianFunctions
reload(xrdmaptools);

c:\Users\emusterma\AppData\Local\anaconda3\envs\sXRDMap-py311-1.0\Lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Connecting to databrokers...failed.


In [26]:
scanid = '156117'
scantype = 'xrd'
base = 'D:\\Musterman_postdoc\\20240610_Musterman\\flat_fields\\'
map_filename = f'scan{scanid}_dexela_{scantype}.tif'

test = XRDMap.from_image_stack(map_filename, wd=f'{base}', energy=10, scanid=scanid, save_hdf=False)

test.set_calibration('scan156112_dexela_calibration.poni', filedir='D:\\Musterman_postdoc\\20240610_Musterman\\calibrations\\')
test.map.dtype = np.float32

Loading images...done!
Setting detector calibration...


In [27]:
dark_dir = 'D:\\Musterman_postdoc\\20240610_Musterman\\dark_fields\\'
dark_id = 156115
dir_mask = [str(dark_id) in d for d in os.listdir(dark_dir)]

dark_field = io.imread(f'{dark_dir}{np.array(os.listdir(dark_dir))[dir_mask][0]}').astype(np.float32)
test.map.correct_dark_field(dark_field=dark_field)

Correcting dark-field...done!


In [21]:
test.map.apply_solidangle_correction(apply=False)
test.map.apply_polarization_correction(apply=False)

In [22]:
test.plot_image(test.map.polarization_correction)

In [23]:
test.plot_image(test.map.med_image / test.map.solidangle_correction / test.map.polarization_correction)

In [28]:
test.plot_image(test.map.med_image)

In [11]:
test.map.rescale_images(upper=100, lower=0, arr_min=0)

In [61]:

scanlist = [156116, 156117, 156118, 156125, 156126]
scanlist = [156120, 156121, 156122, 156123, 156124]
scanlist = [156128, 156129, 156130, 156131, 156132]
scanlist = [156134, 156135, 156136, 156137, 156138]
scanlist = [156140, 156141, 156142, 156143, 156144]
scanlist = [156146, 156147, 156148, 156149, 156150]
xrdmaps = []

dark_dir = 'D:\\Musterman_postdoc\\20240610_Musterman\\dark_fields\\'
dark_id = 156145
dir_mask = [str(dark_id) in d for d in os.listdir(dark_dir)]

dark_field = io.imread(f'{dark_dir}{np.array(os.listdir(dark_dir))[dir_mask][0]}').astype(np.float32)


for scanid in scanlist:
    scantype = 'xrd'
    base = 'D:\\Musterman_postdoc\\20240610_Musterman\\flat_fields\\'
    map_filename = f'scan{scanid}_dexela_{scantype}.tif'

    test = XRDMap.from_image_stack(map_filename, wd=f'{base}', energy=10, scanid=scanid, save_hdf=False)

    test.set_calibration('scan156112_dexela_calibration.poni', filedir='D:\\Musterman_postdoc\\20240610_Musterman\\calibrations\\')
    test.map.dtype = np.float32

    test.map.correct_dark_field(dark_field=dark_field)

    xrdmaps.append(test)

Loading images...done!
Setting detector calibration...
Calibration performed under different settings. Adjusting calibration.
Correcting dark-field...done!
Loading images...done!
Setting detector calibration...
Calibration performed under different settings. Adjusting calibration.
Correcting dark-field...done!
Loading images...done!
Setting detector calibration...
Calibration performed under different settings. Adjusting calibration.
Correcting dark-field...done!
Loading images...done!
Setting detector calibration...
Calibration performed under different settings. Adjusting calibration.
Correcting dark-field...done!
Loading images...done!
Setting detector calibration...
Calibration performed under different settings. Adjusting calibration.
Correcting dark-field...done!


In [62]:
all_images = np.vstack([xrdmap.map.images for xrdmap in xrdmaps])

In [63]:
test.plot_image(np.median(all_images, axis=(0, 1)))

In [60]:
io.imsave(base + 'flat_field_1x1_hr_med.tif', np.median(all_images, axis=(0, 1)))
io.imsave(base + 'flat_field_1x1_hr_min.tif', np.min(all_images, axis=(0, 1)))

In [129]:
test = XRDMap.from_hdf('scan156853_xrd.h5', wd='D:\\Musterman_postdoc\\20240610_Musterman\\xrdmaps\\', save_hdf=False)
dark_field = io.imread(f'{dark_dir}/scan156858_dexela_median_composite.tif')
flat_field = io.imread(f'{base}/flat_field_4x4_hr_med.tif')
test.map.correct_dark_field(dark_field=dark_field)

Loading data from hdf file...
Loading most recent images (raw_images)...done!
Loading scalers...done!
Loading positions...done!
Instantiating ImageMap...done!
XRD Map loaded!
Correcting dark-field...done!


In [130]:
test.set_calibration('scan156160_dexela_calibration.poni', filedir='D:\\Musterman_postdoc\\20240610_Musterman\\calibrations\\')

Setting detector calibration...


In [81]:
test.plot_image(test.map.med_image / flat_field )

In [82]:
test.plot_image(test.map.med_image / flat_field - jelly)

In [78]:
jelly = test.map.med_image / flat_field

In [75]:
air_scatter = test.map.med_image / flat_field

In [85]:
test.tth_resolution = 0.01

In [86]:
tth, intensity = test.integrate1d_image(test.map.med_image / flat_field - jelly)

In [90]:
test.plot_integration(test.integrate1d_image(test.map.med_image / flat_field - air_scatter)[1], tth=test.tth)

In [111]:
tth, intensity = test.integrate1d_image((test.map.med_image / flat_field - jelly) * np.sin(np.radians(test.tth_arr)), tth_num=5000)
test.plot_integration(intensity, tth=tth)

In [167]:
tth, intensity = test.integrate1d_image((test.map.med_image / flat_field - jelly), tth_num=10000)

fig, ax = plt.subplots(1,1, figsize=(5, 5), dpi=200)

ax.plot(tth_2_q(tth, wavelength=test.wavelength), intensity)

ax.set_xlabel('scattering vector (A-1)')

fig.show()

In [189]:
from scipy.ndimage import gaussian_filter
from scipy.fft import fft, fftfreq
from xrdmaptools.utilities.math import tth_2_q, convert_qd

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

N = int(1e7)
yf = fft(intensity, n=N)
xf = fftfreq(N, np.mean(np.diff(tth_2_q(tth, wavelength=test.wavelength))))[:N//2]

ax.plot(xf, 2.0 / N * np.abs(yf[0 : N//2]))

fig.show()